# Model Ensemble 練習

### 本節大綱
* [Ensemble 1. Vote](#Ensemble-1.-Vote)
* [Ensemble 2. Stacking](#Ensemble-2.-Stacking)
* [Ensemble 3. Test Time Augmentation](#Ensemble-3.-Test-Time-Augmentation)

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Flatten, Activation
import tensorflow as tf
import tensorflow.keras.backend as K

## Where AM I 資料讀入及前處理

In [4]:
data_path = "/datasets/aia-data2020/where_am_i/train"

x_data_list = []
y_data_list = []
for roots, dirs, files in os.walk(data_path):
    for each in files:
        if each.find('checkpoint') == -1:
            x_data_list.append(os.path.join(roots.split("/")[-1], each))
            y_data_list.append(roots.split("/")[-1])

In [5]:
data_list = pd.DataFrame({})
data_list['img_path'] = x_data_list
data_list['label'] = y_data_list

In [6]:
data_list.head(3)

img_path    label
0  highway/image_0048.jpg  highway
1  highway/image_0082.jpg  highway
2  highway/image_0112.jpg  highway

In [37]:
from sklearn.model_selection import train_test_split

train_list, test_list = train_test_split(data_list,
                                         test_size=0.1,
                                         random_state=42,
                                         stratify=data_list['label'].values)

## 讀入 Model
<div align=left><img src="./slide_image/Ensemble1.png" width=700  /> 

In [38]:
num_class = len(data_list['label'].unique())
batch_size = 32

In [39]:
model_list = ['vgg16', 'inceptionV3', 'resnet50']

In [40]:
def model_prediction(model_name, datalist):
    model_folder = "./model-logs"
    model_file = "basic_model-best-model.h5"
    model_path = os.path.join(model_folder, model_name, model_file)

    if model_name == 'vgg16':
        from tensorflow.keras.applications.vgg16 import preprocess_input
        img_size = 224
    elif model_name == 'inceptionV3':
        from tensorflow.keras.applications.inception_v3 import preprocess_input
        img_size = 299
    elif model_name == 'resnet50':
        from tensorflow.keras.applications.resnet import preprocess_input
        img_size = 224

    img_shape = (img_size, img_size)

    data_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    pred_generator = data_datagen.flow_from_dataframe(dataframe=datalist,
                                                      directory=data_path,
                                                      x_col="img_path",
                                                      y_col="label",
                                                      target_size=img_shape,
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      shuffle=False)
    model = load_model(model_path)
    print(f'Model is {model_name}.')

    result = model.predict_generator(pred_generator)

    loss, acc = model.evaluate_generator(pred_generator, verbose=2)
    print(f'Accuracy is {acc}.')
    print('-'*50)

    return result, pred_generator.class_indices

In [41]:
all_train_pred = {}
all_test_pred = {}
all_class = {}

for each in model_list:
    all_train_pred[each], all_class[each] = model_prediction(each, train_list)
    K.clear_session()

for each in model_list:
    all_test_pred[each], _ = model_prediction(each, test_list)
    K.clear_session()

Found 2686 validated image filenames belonging to 15 classes.


OSError: SavedModel file does not exist at: ./model-logs/vgg16/basic_model-best-model.h5/{saved_model.pbtxt|saved_model.pb}

In [21]:
for i in range(len(model_list)-1):
    name1 = model_list[i]
    name2 = model_list[i+1]
    print(f'{all_class[name1] == all_class[name2]}')

KeyError: 'vgg16'

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [23]:
y_train = train_list['label'].map(all_class['vgg16']).values
y_test = test_list['label'].map(all_class['vgg16']).values

KeyError: 'vgg16'

## Ensemble 1. Vote
* Mode 眾數
* Probability Mean 機率平均值

* ### Mode 眾數
<img src="./slide_image/Ensemble2.png" width=700  /> 

In [24]:
test_vote = np.zeros((len(test_list), num_class))

for each in model_list:
    test_pred = all_test_pred[each].argmax(-1)
    test_vote += np.eye(num_class)[test_pred]

test_vote = test_vote.argmax(-1)

KeyError: 'vgg16'

In [25]:
print(accuracy_score(y_test, test_vote))

NameError: name 'y_test' is not defined

* ### Probability Mean 機率平均值
<img src="./slide_image/Ensemble3.png" width=700  /> 

In [26]:
test_pred_prob = [all_test_pred[each] for each in model_list]
test_pred_prob = np.array(test_pred_prob).transpose((1, 2, 0))

KeyError: 'vgg16'

In [27]:
test_pred_prob = test_pred_prob.mean(-1)

NameError: name 'test_pred_prob' is not defined

In [28]:
y_pred = test_pred_prob.argmax(-1)

NameError: name 'test_pred_prob' is not defined

In [29]:
accuracy_score(np.abs(np.around(y_pred, decimals=0)), y_test)

NameError: name 'y_pred' is not defined

## Ensemble 2. Stacking

-- 直接對 3(models) 預測的結果當作線性模型的輸入，並且給予相對應的加權權重<br>
-- 最後以加權後的結果所對應到的類別，當作最終所屬類別

<div align=left><img src="./slide_image/Ensemble4.png" width=700  /> 

In [30]:
train_pred_prob = [all_train_pred[each] for each in model_list]
train_pred_prob = np.array(train_pred_prob).transpose((1, 2, 0))
train_pred_prob = train_pred_prob.astype('float32')

KeyError: 'vgg16'

In [31]:
test_pred_prob = [all_test_pred[each] for each in model_list]
test_pred_prob = np.array(test_pred_prob).transpose((1, 2, 0))
test_pred_prob = test_pred_prob.astype('float32')

KeyError: 'vgg16'

In [84]:
inputs = Input(shape=(15, 3))
x = Conv1D(1, 1)(inputs)
x = Flatten()(x)
outputs = Activation('softmax')(x)

stack_model1 = Model(inputs=inputs, outputs=outputs)

In [85]:
learning_rate = 1e-6
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

stack_model1.compile(loss='categorical_crossentropy',
                     optimizer=optimizer,
                     metrics=['accuracy'])

In [86]:
batch_size = 32
epochs = 10
y_train_1H = np.eye(15, dtype='float32')[y_train]
history = stack_model1.fit(train_pred_prob, y_train_1H,
                           batch_size=batch_size,
                           epochs=epochs,)

Train on 2686 samples
Epoch 1/10
2686/2686 [==============================] - 1s 195us/sample - loss: 2.2386 - accuracy: 0.9899
Epoch 2/10
2686/2686 [==============================] - 0s 91us/sample - loss: 2.2384 - accuracy: 0.9899
Epoch 3/10
2686/2686 [==============================] - 0s 93us/sample - loss: 2.2381 - accuracy: 0.9899
Epoch 4/10
2686/2686 [==============================] - 0s 95us/sample - loss: 2.2379 - accuracy: 0.9899
Epoch 5/10
2686/2686 [==============================] - 0s 90us/sample - loss: 2.2377 - accuracy: 0.9899
Epoch 6/10
2686/2686 [==============================] - 0s 94us/sample - loss: 2.2375 - accuracy: 0.9899
Epoch 7/10
2686/2686 [==============================] - 0s 91us/sample - loss: 2.2373 - accuracy: 0.9899
Epoch 8/10
2686/2686 [==============================] - 0s 95us/sample - loss: 2.2371 - accuracy: 0.9899
Epoch 9/10
2686/2686 [==============================] - 0s 92us/sample - loss: 2.2369 - accuracy: 0.9899
Epoch 10/10
2686/2686 [=========

In [87]:
y_test_1H = np.eye(15, dtype='float32')[y_test]
stack_model1.evaluate(test_pred_prob, y_test_1H, verbose=2)

299/1 - 0s - loss: 2.2802 - accuracy: 0.9365


[2.274052445306427, 0.93645483]

In [81]:
K.clear_session()

## Ensemble 3. Test Time Augmentation
<div align=left><img src="./slide_image/Ensemble5.png" width=700  /> 

In [32]:
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [33]:
import cv2

In [43]:
def pre_flip(img):
    return cv2.flip(img, 1)

def pre_shift(img, shift_range=0.1, direct='r'):
    h, w, _ = img.shape
    tx = int(img.shape[1] * shift_range)   # 向右 tx
    ty = 0                                 # 向下 ty
    if direct == 'r':
        M1 = np.float32([[1, 0, tx],
                         [0, 1, ty]])
    elif direct == 'l':
        M1 = np.float32([[1, 0, -tx],
                         [0, 1, ty]])

    return cv2.warpAffine(img, M1, (w, h))

In [44]:
def tta_generator(dataframe, directory, x_col, y_col, target_size):
    col_label = dataframe[y_col].map(class_name).values
    while True:
        for ipath, ilabel in zip(dataframe[x_col].values, col_label):
            img_path = os.path.join(directory, ipath)
            tta_img = []
            img = cv2.imread(img_path)
            img = cv2.resize(img, (target_size))
            img = preprocess_input(img)

            tta_img.append(img)
            tta_img.append(pre_flip(img))
            tta_img.append(pre_shift(img, shift_range=0.01, direct='r'))
#             tta_img.append(pre_shift(img, shift_range=0.01, direct='l'))

            yield np.array(tta_img).astype('float32'), ilabel

In [45]:
class_name = all_class[name1]

In [46]:
img_size = 299
my_tta = tta_generator(test_list,
                       data_path,
                       x_col="img_path",
                       y_col="label",
                       target_size=(img_size, img_size))

In [47]:
model_folder = "./model-logs"
model_file = "basic_model-best-model.h5"
model_name = "inceptionV3"
model_path = os.path.join(model_folder, model_name, model_file)

In [48]:
model = load_model(model_path)

In [49]:
from scipy.stats import mode

In [50]:
test_tta_true = []
test_tta_pred = []
for _ in range(len(test_list)):
    imgs, labels = next(my_tta)
    tmp = model.predict(imgs)
    test_tta_pred.append(mode(tmp)[0])
    test_tta_true.append(labels)

test_tta_true = np.array(test_tta_true).reshape(-1)

test_tta_pred = np.array(test_tta_pred).argmax(-1)
test_tta_pred = test_tta_pred.reshape(-1)

In [51]:
accuracy_score(test_tta_pred, test_tta_true)

0.919732441471572